In [ ]:
import os, re, matplotlib, pandas, collections, importlib, sys, pickle, random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.stats import norm
#from sklearn.neighbors import KernelDensity
#from scipy.signal import savgol_filter

sys.path.append('/Users/dp/pma/')

from importlib import reload
import sameRiver
#from sameRiver import *
import sameRiver.scheme

top = top_dir = '/Users/dp/pma/dataAndScripts/clip/old_mapping_meta/'
os.chdir(top)


def numeric_columns(_df):
    return [x for x in _df.columns if (_df[x].dtype.kind in 'bifc')]


figname = lambda _str: '/Users/dp/pma/dataAndScripts/clip/figs/{}.pdf'.format(_str)

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rc('font', **{'family':'sans-serif','sans-serif':['Arial']})


matplotlib.rcParams['axes.linewidth'] = 0.3 
matplotlib.rcParams['xtick.major.width'] = 0.8

In [ ]:
def multiply_by_xl_rate(df):
    xl_rate = '/Users/dp/pma/percentCrosslinked.xlsx'

    xl = pandas.read_excel(xl_rate)
    to_xl = dict(xl.groupby('Protein').mean())
    no_space = {}
    
    for k, v in to_xl.items():
        no_space[re.sub(' ', '', k)] = v
    to_xl.update(no_space)
    
    by_rate = df.copy()
    for col in df.columns:

        if col == 'gene_name' or col == 'Gene type':
            continue
        gene = col.split('_')[1]

        if gene in to_xl: # Per billion -> 10 * to_xl[gene] * _df[col]
            by_rate[col] = 10 * to_xl[gene] * df[col]
        else:
            del by_rate[col]

    #print(by_rate.iloc[:2, :2])

    return by_rate

def per_million(df):
    _df = df.copy()
    for col in [_ for _ in df.columns if df[_].dtype.kind in 'bifc']:
        _sum = _df[col].sum()
        _df[col] = 1E6 * _df[col]/_sum
    return _df

def norm_to_per_cell(df):
    exp = pandas.read_excel('/Users/dp/pma/dataAndScripts/tables/lysateExpression.xlsx')

    to_expression = dict(exp.groupby('Protein')['Value'].mean())
    
    no_space = {}
    for k, v in to_expression.items():
        ns = re.sub(' ', '', k)
        no_space[ns] = v
        no_space[re.sub('L100', '100', ns)] = v
    to_expression.update(no_space)
    
    by_cell = df.copy()
    for col in df.columns:

        if col in ['gene_name', 'Gene name', 'gene', 'Gene type']:
            continue
            
        gene = col.split('_')[1]
        gene = re.sub(' ', '', gene)
        gene = re.sub('L100', '100', gene)
        if gene not in to_expression:
            del by_cell[col]
            continue
            
        by_cell[col] = to_expression[gene] * df[col]
    return by_cell
        
#per_million(df)
#arrays = make_gauss_figure(by_rate)

In [ ]:
def get_params(gene):
    if gene in ['ITPA', 'CDK4',
                'UBA2', 
                'ETS2', 'DCNT6', 'IDE', 'VPS24', 'CAPNS6', 'TPGS2', 
                'EPB41L5', 
                'CCIN',
                ]:
        color = 'k'
        alpha = 0.2
        label = 'Random protein'
    elif gene in ['FBL', 'hnRNPC', 'SF3B1', 'PCBP1', 'CELF1', 'hnRNPD', 'Rbfox1']:
        #, 'PCBP1 L100P', 'PCBP1 L100Q', 'PCBP1']:
        color = 'r'
        alpha = 0.4
        label = 'RBP'
    elif gene in ['No vector']:
        color = 'b'
        alpha = 0.2
        label = 'No vector'
    else:
        color = 'g'
        alpha = 0.1
        label = 'Other'
    return (color, alpha, label)

In [ ]:
mpl=matplotlib
mpl.rcParams#['axes.prop_cycle']

In [ ]:
??pandas.DataFrame.clip(lower=0)

In [ ]:
def smooth(y, box_pts):
    #https://stackoverflow.com/questions/20618804/how-to-smooth-a-curve-in-the-right-way
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def make_hist(df, suffix):
    
    # Initializations.
    fig, ax = plt.subplots()

    added_labels = set()
    genes_plotted = set()
    arr2d = None
    to_plot = []

    # For each protein.
    for protein in numeric_columns(df):

        (color, alpha, label) = get_params(protein)
            
        linestyle = '-'
        
        pre_clipping = np.nan_to_num(df[protein])
        
        if protein in genes_plotted:
            print('????')
            continue
            
        genes_plotted.add(protein)
#        print(df[col][df[col]>1E4])

        raw_reads = np.clip(pre_clipping, 0, 1E6)
        raw_reads = raw_reads[raw_reads>0]

        bins = 10
        (vals, edges, patches) = plt.hist(np.log10(raw_reads), bins=bins)

        y = smooth(vals, bins)
        #print(len(edges[:-1]), len(y), '<<< smoothed lens')
        
        #plt.bar(edges[:-1], y, width=(edges[1]-edges[0]))
        to_plot.append((edges[:-1], vals, re.sub(':', ' ', protein)))
        
    plt.clf()
    RBPs = ['Rbfox1', 'hnRNPC', 'FBL', 'CELF1', 'PCBP1', 'hnRNPD', ]
    other_rbps = []#'A1CF', 'KHDRBS2', 'FUBP1']#, 'PCBP1 ∆KH2']
    randos = ['ITPA', 'CDK4', 'UBA2', 'ETS2', 'DCNT6', 'IDE', 'CHMP3', 'TPGS2',] #'EPB41L5', #'CCIN',
                
    colors = sns.cubehelix_palette(len(RBPs), start=.5, rot=-2)
    for n, (x, y, gene) in enumerate(to_plot):
        (color, alpha, label) = get_params(gene)
        marker='o'
        if gene in RBPs:
            linestyle = '-'
            color = colors[RBPs.index(gene)]
            alpha = 1
            linewidth = 0.7
            marker = 'o'
            markersize = 3
        elif gene in randos:
            linestyle = ':'
            color = 'k'
            alpha = 0.2
            linewidth = 0.5
            marker = ''
            markersize = 2
        elif gene in other_rbps:
            linestyle = ':'
            color = 'r'
            alpha = 0.2
            linewidth = 0.5
            marker = ''
            markersize = 2
        else:
            continue
        plt.plot(x, y, c=color, label=re.sub('[:-]', ' ', gene), markersize=markersize, linestyle=linestyle, 
                 marker=marker, linewidth=linewidth, alpha=alpha)
    #plt.legend()
    plt.title(suffix)
    plt.xlabel("Frequency of interaction (log10 reads per million)")
    #plt.xlabel('Frequency of XL (per billion proteins, log10)')
    plt.ylabel("# RNAs")
    fig.set_figwidth(3.6)  # 1.2
    fig.set_figheight(2.4)  # 0.8
    #[i.set_linewidth(0.1) for i in ax.spines.itervalues()]
    fig.savefig('/Users/dp/pma/dataAndScripts/clip/figs/hist_smooth_xls_per_protein_{}.pdf'.format(suffix))
    plt.show()
    plt.clf()
            
    #print(arr2d)
    #sns.heatmap(np.log10(arr2d))
    #plt.show()
    #plt.clf()
    
# Load counts data.
scheme = sameRiver.scheme.scheme(f'{top}/scheme.xlsx')
import sameRiver.readsPerGene
from sameRiver.readsPerGene import *

rpg = readsPerGene(f"{top}/ann_counts.txt", scheme_filename=f"{top}/scheme.xlsx")
rpm = readsPerMillion(rpg)
xpp = xlsPerProtein(rpm, xl_rate_fname='/Users/dp/pma/percentCrosslinked.xlsx')

df = rpm.average_by_protein()
df['Gene type'] = [rpm.df.loc[x, 'Gene type'] for x in df.index]
df.columns = [re.sub('[:-]', ' ', x) for x in df.columns]
print(df.head())

xl = xpp.average_by_protein()
xl['Gene type'] = [xpp.df.loc[x, 'Gene type'] for x in xl.index]
xl.columns = [re.sub('[:-]', ' ', x) for x in xl.columns]

# Convert to per cell (relative).
#per_cell = norm_to_per_cell(xl)

# Convert some pseudo-snRNA genes from snRNA to pseudo_snRNA.
def snRNAs(name, _type):
    if _type == 'snRNA':
        if re.search('\d+P', name):
            return 'pseudo_snRNA'
        else:
            return 'snRNA'
    else:
        return _type

df['Gene type'] = [snRNAs(name, _type) for name, _type in \
                  zip(df.index, df['Gene type'])]

# Make all of the plots.
def plot_em(prefix, df):
            
    all_rna = make_hist(df.copy(), prefix + 'all_RNA')
    
    mRNA = df[df['Gene type']=='protein_coding'].copy()
    mRNA_arrays_to_plot = make_hist(mRNA, prefix + 'mRNA')

    ncRNA = df[df['Gene type']=='snRNA'].copy()
    ncRNA_arrays_to_plot = make_hist(ncRNA, prefix + 'snRNA')    

    snoRNA = df[df['Gene type']=='snoRNA'].copy()
    snoRNA_arrays_to_plot = make_hist(snoRNA, prefix + 'snoRNA')

    ncRNA = df[df['Gene type']!='protein_coding'].copy()
    ncRNA_arrays_to_plot = make_hist(ncRNA, prefix + 'ncRNA')
    
    ncRNA = df[df['Gene type']=='tRNA'].copy()
    ncRNA_arrays_to_plot = make_hist(ncRNA, prefix + 'tRNA')

plot_em('RBPs_per_protein_', xl)
plot_em('RBPs_per_XL_', df)



In [ ]:

def make_hist(df, suffix):
    
    # Initializations.
    fig, ax = plt.subplots()

    added_labels = set()
    genes_plotted = set()
    arr2d = None
    to_plot = []

    # For each protein.
    for protein in numeric_columns(df):
        #print(protein)

        (color, alpha, label) = get_params(protein)
            
        linestyle = '-'
        
        pre_clipping = np.nan_to_num(df[protein])
        
        if protein in genes_plotted:
            print('????')
            continue
            
        genes_plotted.add(protein)
#        print(df[col][df[col]>1E4])

        raw_reads = np.clip(pre_clipping, 0, 1E7)
        raw_reads = raw_reads[raw_reads>0]

        bins = 10
        (vals, edges, patches) = plt.hist(np.log10(raw_reads), bins=bins)

        y = smooth(vals, bins)
        #print(len(edges[:-1]), len(y), '<<< smoothed lens')
        
        #plt.bar(edges[:-1], y, width=(edges[1]-edges[0]))
        to_plot.append((edges[:-1], vals, re.sub(':', ' ', protein)))
        
    plt.clf()
    RBPs = ['hnRNPC', 'FBL', 'hnRNPD', 'CELF1', 'Rbfox1'] #, 'A1CF', 
           #'KHDRBS2', ]
    pcbp1 = ['PCBP1', 'PCBP1 100P', 'PCBP1 100Q', 'PCBP1 dKH']
    randos = ['ITPA', 'CDK4', 'UBA2', 
                'ETS2', 'DCNT6', 'IDE', 'CHMP3', #'CAPNS6', 
                'TPGS2']  #'EPB41L5', #'CCIN',
                
    colors = sns.cubehelix_palette(len(pcbp1), start=.5, rot=-2)
    for n, (x, y, gene) in enumerate(to_plot):
        (color, alpha, label) = get_params(gene)
        marker='o'
        if gene in pcbp1:
            linestyle = '-'
            color = colors[pcbp1.index(gene)]
            alpha = 1
            linewidth = 0.7
            marker = 'o'
            markersize = 3
        elif gene in randos:
            linestyle = ':'
            color = 'k'
            alpha = 0.2
            linewidth = 0.5
            marker = ''
            markersize = 2
        elif gene in RBPs:
            linestyle = ':'
            color = 'r'
            alpha = 0.2
            linewidth = 0.5
            marker = ''
            markersize = 2
        else:
            continue
        plt.plot(x, y, c=color, label=re.sub('[:-]', ' ', gene), markersize=markersize, linestyle=linestyle, marker=marker, linewidth=linewidth, alpha=alpha)
    plt.legend()
    plt.title(suffix)
    plt.xlabel("Frequency of interaction (log10 reads per million)")
    #plt.xlabel('Frequency of XL (per billion proteins, log10)')
    plt.ylabel("# RNAs")
    fig.set_figwidth(3.6)  # 1.2
    fig.set_figheight(2.4)  # 0.8
    fig.savefig('/Users/dp/pma/dataAndScripts/clip/figs/hist_smooth_xls_per_protein_{}.pdf'.format(suffix))
    plt.show()
    plt.clf()
            
    #print(arr2d)
    #sns.heatmap(np.log10(arr2d))
    #plt.show()
    #plt.clf()
    
# Make all of the plots.
def plot_em(prefix, df):
    
    all_rna = make_hist(df.copy(), prefix + 'all_RNA')
    
    mRNA = df[df['Gene type']=='protein_coding'].copy()
    mRNA_arrays_to_plot = make_hist(mRNA, prefix + 'mRNA')

    ncRNA = df[df['Gene type']=='snRNA'].copy()
    ncRNA_arrays_to_plot = make_hist(ncRNA, prefix + 'snRNA')    

    snoRNA = df[df['Gene type']=='snoRNA'].copy()
    snoRNA_arrays_to_plot = make_hist(snoRNA, prefix + 'snoRNA')

    ncRNA = df[df['Gene type']!='protein_coding'].copy()
    ncRNA_arrays_to_plot = make_hist(ncRNA, prefix + 'ncRNA')
    
    ncRNA = df[df['Gene type']=='tRNA'].copy()
    ncRNA_arrays_to_plot = make_hist(ncRNA, prefix + 'tRNA')

plot_em('PCBP1_per_XL_', df)
plot_em('PCBP1_per_protein_', xl)
